In [3]:
import sys
sys.path.append("../") 
from infer_veritas import  *
from truthfinder import *

import dill as pickle
import gzip

In [9]:
with gzip.open('tmp_veritas_001_global.pkl.gz', 'rb') as file:
    TR=pickle.load(file)


In [10]:
TR.veritas_model

{'version': '0.0.1', 'model': qnet.Qnet, 'model_neg': qnet.Qnet, 'problem': ''}

In [11]:
M=TR.veritas_model

In [12]:
M['upper_scoretoprobability'](.8)


KeyError: 'upper_scoretoprobability'

In [ ]:
pred=M['calibration_df'].upper.values
len(pred)

In [ ]:
df_test = TR.data.loc[[x for x in TR.data.index.values
                       if x not in TR.training_index],:][np.array(
                               list(TR.veritas_model['model']\
                                    .feature_names)+[TR.target_label])]
featurenames=df_test.drop(TR.target_label, axis=1).columns
adictn=df_test[featurenames[TR.shap_index[:TR.query_limit]]].T.to_dict()
adictn=[{key:value} for key,value in adictn.items()]

keyindex=[list(x.keys())[0] for x in adictn]
labels = df_test.loc[keyindex,TR.target_label]#.values
lf=pd.DataFrame(labels)

In [ ]:
lf=lf[lf!=''].dropna()

In [ ]:
xf=pd.DataFrame(pred,lf.values).reset_index().replace('',pd.NA).dropna()

In [ ]:
xf.columns=['dx','pred']
xf.dx=[x[0] for x in xf.dx]
xf

In [ ]:
xf.dx=xf.dx.astype(int)
fpr, tpr, thresholds = metrics.roc_curve(xf.dx, xf.pred, pos_label=1)
import pylab as plt
plt.plot(fpr,tpr)
rf=pd.DataFrame(tpr,fpr,columns=['tpr']).assign(threshold=thresholds)
rf.index.name='fpr'
zt=zedstat.processRoc(df=rf.reset_index(),
                      order=3, 
                      total_samples=1000,
                      positive_samples=500,
                      alpha=0.01,
                      prevalence=0.5)
zt.smooth(STEP=0.001)
zt.allmeasures(interpolate=True)
zt.usample(precision=3)
Z=zt.get()

In [ ]:
zt.auc()

In [ ]:
zt.scoretoprobability(1.3)

In [ ]:
rf